In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **1. Data Cleaning and Preprocessing**

In [8]:
import pandas as pd

csvfile = '/kaggle/input/housing-prices-dataset/Housing.csv'
data = pd.read_csv(csvfile)

print(data.head())

print(data.isnull().sum())

data = data.dropna()

data = pd.get_dummies(data, drop_first=True)

print(data.head())

      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  
price               0
area                0
bedrooms            0
bathrooms           0
stories    

# **2. Feature Engineering**

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerical_features = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
data[numerical_features] = scaler.fit_transform(data[numerical_features])

print(data.head())

      price      area  bedrooms  bathrooms   stories   parking  mainroad_yes  \
0  13300000  1.046726  1.403419   1.421812  1.378217  1.517692          True   
1  12250000  1.757010  1.403419   5.405809  2.532024  2.679409          True   
2  12250000  2.218232  0.047278   1.421812  0.224410  1.517692          True   
3  12215000  1.083624  1.403419   1.421812  0.224410  2.679409          True   
4  11410000  1.046726  1.403419  -0.570187  0.224410  1.517692          True   

   guestroom_yes  basement_yes  hotwaterheating_yes  airconditioning_yes  \
0          False         False                False                 True   
1          False         False                False                 True   
2          False          True                False                False   
3          False          True                False                 True   
4           True          True                False                 True   

   prefarea_yes  furnishingstatus_semi-furnished  furnishingst

# **3. Model Selection and Training**

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = data.drop('price', axis=1)
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

# **4. Model Evaluation and Fine-Tuning**

In [11]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

train_rmse = mean_squared_error(y_train, y_pred_train, squared=False)
test_rmse = mean_squared_error(y_test, y_pred_test, squared=False)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"Training RMSE: {train_rmse}")
print(f"Testing RMSE: {test_rmse}")
print(f"Training R2: {train_r2}")
print(f"Testing R2: {test_r2}")

Training RMSE: 984051.9236507412
Testing RMSE: 1324506.9600914405
Training R2: 0.6859438988560158
Testing R2: 0.6529242642153175


# **5. Fine-Tuning**

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

rf_model = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

gridsearch = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
gridsearch.fit(X_train, y_train)

best_model = gridsearch.best_estimator_

y_pred_test_best = best_model.predict(X_test)
best_test_rmse = mean_squared_error(y_test, y_pred_test_best, squared=False)
best_test_r2 = r2_score(y_test, y_pred_test_best)

print(f"Best Testing RMSE: {best_test_rmse}")
print(f"Best Testing R2: {best_test_r2}")

Best Testing RMSE: 1448581.4316019295
Best Testing R2: 0.5848532987560832
